In [34]:
%pip install -r requirements.txt --upgrade


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 846.5/846.5 kB 10.0 MB/s eta 0:00:00 MB/s eta 0:00:01
  Using cached tiktoken-0.4.0-cp310-cp310-macosx_11_0_arm64.whl (761 kB)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 389.5/389.5 kB 57.3 MB/s eta 0:00:00
  Attempting uninstall: langchain
    Found existing installation: langchain 0.0.170
    Uninstalling langchain-0.0.170:
      Successfully uninstalled langchain-0.0.170
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gpt-index 0.4.1 requires tenacity<8.2.0, but you have tenacity 8.2.1 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [ ]:
import whisper
model = whisper.load_model("medium.en")

In [35]:
from dotenv import load_dotenv

load_dotenv()

True

In [3]:
import yt_dlp

URLS = []

ydl_opts = {
    'format': 'm4a/bestaudio/best',
    'postprocessors': [{
        'key': 'FFmpegExtractAudio',
        'preferredcodec': 'm4a',
    }],
    'outtmpl': './tmp/foo_%(title)s-%(id)s.%(ext)s'
}

with yt_dlp.YoutubeDL(ydl_opts) as ydl:
    error_code = ydl.download(URLS)

In [4]:
import os
import urllib.parse

podcasts_to_analyze = {}

for file in os.listdir("./tmp"):
    file_path = os.path.join("./tmp", file)
    transcript_name = f"{urllib.parse.quote(file)}.txt"
    
    transcript_file_path = os.path.join("./podcasts-raw-transcripts", transcript_name)
    
    if not os.path.exists(transcript_file_path):
        print(transcript_file_path)
        result = model.transcribe(file_path)
        podcasts_to_analyze[file] = result["text"]
        
        with open(f"./podcasts-raw-transcripts/{transcript_name}", "w") as f:
            f.write(result["text"])
    else:
        print(f"Skipping {file_path}")

Skipping ./tmp/Latent Space Podcast - Episode 10 final mix v1.flac


In [5]:
from langchain import PromptTemplate


template = """
Read the transcript of the podcast below:
 {podcast_contents}

Create bullet points with the main topics of the podcast, followed by the opinions of the speakers. 
"""

podcast_summary_prompt = PromptTemplate(
    input_variables=["podcast_contents"],
    template=template,
)


In [6]:
from langchain import PromptTemplate


template = """
When I listen to a podcast, I take notes on the main talking points of the hosts. I divide it in sections based on topics discussed. 
If the host mentions a specific technology or product, I note that in double brackets like this: [[artificial intelligence]].

These are the notes from the last podcast I listened to:

{podcast_notes}

Write a {words_count} words summary of the notes.
"""

notes_summary_template = PromptTemplate(
    input_variables=["podcast_notes", "words_count"],
    template=template,
)


In [7]:
from langchain import PromptTemplate


template = """
When I take notes for a podcast, I like to also write twitter threads to share them. Each tweet should end saying how far we are in the thread; if it's a 5 tweets thread, the first tweet should end with (1/5), the second one with (2/5), etc.

The tweets have to be easy to read and catch people's attention. Each of them should include an emoji.

These are the notes from my last podcast:

{podcast_notes}

Create a twitter thread for it.
"""

twitter_thread_template = PromptTemplate(
    input_variables=["podcast_notes"],
    template=template,
)


In [22]:
from langchain.chat_models import ChatAnthropic
from langchain.text_splitter import SpacyTextSplitter
from langchain.prompts.chat import (
    ChatPromptTemplate,
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
)

chat = ChatAnthropic(model='claude-v1-100k')

template="You are hired to create a readable podcast transcript. Do not make any changes to the contents, return the content word by word. Fix things such as capitalization, commas, etc. Separate what the host says from what the guest says. For example: 'what is your name it is Alex' would become 'Host: What is your name? Guest: It is Alex'. The guest name for this episode is {guest_name}."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template="This is the full transcript: {text}"
human_message_prompt = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt = ChatPromptTemplate.from_messages([system_message_prompt, human_message_prompt])

all_results = []


transcripts_folder = "./podcasts-raw-transcripts"
for file in os.listdir(transcripts_folder):
  file_path = os.path.join(transcripts_folder, file)
  
  podcast = open(file_path, "r").read()  
  
  result = chat(chat_prompt.format_prompt(guest_name="Shreya", text=podcast).to_messages()).content

  with open(f"./podcasts-clean-transcripts/{transcript_name}", "w") as f:    
    f.write(result)

In [31]:
import anthropic

client = anthropic.Client(api_key=os.getenv("ANTHROPIC_API_KEY"))

template="You are hired to create a readable podcast transcript. Do not make any changes to the contents, return the content word by word. Fix things such as capitalization, commas, etc. Separate what the host says from what the guest says. For example: 'what is your name it is Alex' would become 'Host: What is your name? Guest: It is Alex'."

transcripts_folder = "./podcasts-raw-transcripts"

result = ""

for file in os.listdir(transcripts_folder):
  file_path = os.path.join(transcripts_folder, file)
  
  podcast = open(file_path, "r").read()  
    
  response = client.completion_stream(
    prompt=f"{anthropic.HUMAN_PROMPT} {template}. This is the full transcript: {podcast} {anthropic.AI_PROMPT}",
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model="claude-v1-100k",
    max_tokens_to_sample=1000,
    stream=True,)
  for data in response:
    print(data)
    result = data

  print(result['completion'])
  
  with open(f"./podcasts-clean-transcripts/{transcript_name}", "w") as f:    
    f.write(result['completion'])

{'completion': ' Here', 'stop': None, 'stop_reason': None, 'truncated': False, 'log_id': 'c0d0d6cfb0cd0374f064a21a07d05af8', 'model': 'claude-v1-100k', 'exception': None}
{'completion': ' Here is', 'stop': None, 'stop_reason': None, 'truncated': False, 'log_id': 'c0d0d6cfb0cd0374f064a21a07d05af8', 'model': 'claude-v1-100k', 'exception': None}
{'completion': ' Here is the', 'stop': None, 'stop_reason': None, 'truncated': False, 'log_id': 'c0d0d6cfb0cd0374f064a21a07d05af8', 'model': 'claude-v1-100k', 'exception': None}
{'completion': ' Here is the transcript formatted', 'stop': None, 'stop_reason': None, 'truncated': False, 'log_id': 'c0d0d6cfb0cd0374f064a21a07d05af8', 'model': 'claude-v1-100k', 'exception': None}
{'completion': ' Here is the transcript formatted with the host', 'stop': None, 'stop_reason': None, 'truncated': False, 'log_id': 'c0d0d6cfb0cd0374f064a21a07d05af8', 'model': 'claude-v1-100k', 'exception': None}
{'completion': " Here is the transcript formatted with the host's

In [32]:
print(result)

{'completion': " Here is the transcript formatted with the host's words in bold and the guest's words in plain text:\n\n**Hey everyone, welcome to the Late in Space podcast. This is Alessio, partner and CTO in Residence at Decibel Partners. I'm joined by my co-host, Wix, writer and editor of Late in Space. And today we have Shreya Rajpal in the studio. Welcome Shreya.**  \n\nHi, hi, excited to be here.   \n\n**Excited to have you too.**  \n\nBut this has been a long time coming, you and I have chatted a little bit and excited to learn more about guardrails. So we do a little intro for you and then we have you fill in the blanks. So you got your bachelor's in IIT Delhi, minor in computer science with focus on AI, which is super relevant now. I bet you didn't think about that in undergrad.  \n\nYeah, I think it's interesting because like I started working in AI back in 2014 and back then I was like, oh, it's here. This is like almost changing the world already. So it feels like, you know

In [36]:
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader

documents = SimpleDirectoryReader('podcasts-clean-transcripts').load_data()
index = GPTVectorStoreIndex.from_documents(documents)
index.storage_context.persist()

INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [build_index_from_nodes] Total embedding token usage: 1206 tokens


In [ ]:
from llama_index import StorageContext, load_index_from_storage

storage_context = StorageContext.from_defaults(persist_dir='./storage')
index = load_index_from_storage(storage_context)

In [37]:

query_engine = index.as_query_engine()
query_engine.query("Who are the podcast hosts?")

INFO:llama_index.token_counter.token_counter:> [retrieve] Total LLM token usage: 0 tokens
INFO:llama_index.token_counter.token_counter:> [retrieve] Total embedding token usage: 6 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total LLM token usage: 1253 tokens
INFO:llama_index.token_counter.token_counter:> [get_response] Total embedding token usage: 0 tokens


Response(response='\nThe podcast hosts are Alessio and Wix.', source_nodes=[NodeWithScore(node=Node(text=" Here is the transcript formatted with the host's words in bold and the guest's words in plain text:\n\n**Hey everyone, welcome to the Late in Space podcast. This is Alessio, partner and CTO in Residence at Decibel Partners. I'm joined by my co-host, Wix, writer and editor of Late in Space. And today we have Shreya Rajpal in the studio. Welcome Shreya.**  \n\nHi, hi, excited to be here.   \n\n**Excited to have you too.**  \n\nBut this has been a long time coming, you and I have chatted a little bit and excited to learn more about guardrails. So we do a little intro for you and then we have you fill in the blanks. So you got your bachelor's in IIT Delhi, minor in computer science with focus on AI, which is super relevant now. I bet you didn't think about that in undergrad.  \n\nYeah, I think it's interesting because like I started working in AI back in 2014 and back then I was like,